In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, cohen_kappa_score
import tensorflow as tf
from tensorflow import keras
from tqdm.auto import tqdm
import time
import nltk
from nltk.corpus import wordnet 
import random
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

nltk.download('wordnet')
print("라이브러리 로드 완료")

# 1. 데이터 로드 및 전처리
print("CSV 파일 로드 중...")
df = pd.read_csv('data/train/train.csv')
print(f"데이터 로드 완료. 샘플 수: {len(df)}")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hope\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


라이브러리 로드 완료
CSV 파일 로드 중...
데이터 로드 완료. 샘플 수: 51630


In [3]:

X = df['사람문장1']
y = df['감정_대분류']

def preprocess_text(text):
    text = text.replace('.', '').replace(',', '')
    words = text.split()
    stopwords = set(['는', '은', '이', '가', '을', '를', '에', '의', '도', '에서', '와', '과', '로', '으로',
                     '그리고', '그러나', '그런데', '하지만', '또는', '그래서', '따라서',
                     '이것', '저것', '그것', '우리', '여러분', '나', '너', '그', '그녀',
                     '입니다', '있습니다', '합니다', '되었습니다', '있다', '것이다', '합니다'])
    return ' '.join([word for word in words if word not in stopwords])

X = X.apply(preprocess_text)

# 2. 데이터 증강
def augment_text(text):
    words = text.split()
    augmented = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            augmented.append(random.choice(synonyms))
        else:
            augmented.append(word)
    return " ".join(augmented)

print("데이터 증강 중...")
augmented_X = [augment_text(text) for text in X]
X = pd.concat([X, pd.Series(augmented_X)])
y = pd.concat([y, y])

# 레이블 인코딩
le = LabelEncoder()
y = le.fit_transform(y)

# 3. BERT 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout_rate=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 4. 학습 함수 정의
def train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device, num_epochs=3):
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            
            # 레이블을 long 타입으로 변환
            labels = labels.long()

            optimizer.zero_grad()
            
            with autocast():
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average training loss: {avg_train_loss:.4f}")
        
        val_accuracy, val_f1 = evaluate_model(model, val_dataloader, device)
        print(f"Validation Accuracy: {val_accuracy:.4f}, F1 Score: {val_f1:.4f}")
    
    return model

# 5. 평가 함수 정의
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch

            # 레이블을 long 타입으로 변환
            labels = labels.long()
            
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    
    accuracy = (np.array(predictions) == np.array(true_labels)).mean()
    f1 = f1_score(true_labels, predictions, average='weighted')
    
    return accuracy, f1

# 6. 메인 학습 루프
def main():
    # BERT 모델 및 토크나이저 로드
    model_name = "klue/bert-base"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 데이터 토큰화
    encoded_data = tokenizer(X.tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoded_data['input_ids']
    attention_mask = encoded_data['attention_mask']
    labels = torch.tensor(y, dtype=torch.long)
    
    # 데이터셋 생성
    dataset = TensorDataset(input_ids, attention_mask, labels)
    
    # K-fold 교차 검증
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"Fold {fold}")
        
        train_dataset = TensorDataset(input_ids[train_idx], attention_mask[train_idx], labels[train_idx])
        val_dataset = TensorDataset(input_ids[val_idx], attention_mask[val_idx], labels[val_idx])
        
        train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
        
        model = BERTClassifier(bert_model, num_classes=len(le.classes_))
        model = model.to(device)
        
        optimizer = AdamW(model.parameters(), lr=2e-5)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * 3)
        
        model = train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, device)
        
        # 모델 저장
        torch.save(model.state_dict(), f'bert_classifier_fold_dataset2_{fold}.pth')
    
    print("Training completed for all folds.")

if __name__ == "__main__":
    main()

데이터 증강 중...


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda
Fold 1


Epoch 1/3:   0%|          | 0/2582 [00:00<?, ?it/s]

Average training loss: 1.1374
Validation Accuracy: 0.6122, F1 Score: 0.6095


...

Average training loss: 0.1569
Validation Accuracy: 0.9950, F1 Score: 0.9950


Epoch 2/3:   0%|          | 0/2582 [00:00<?, ?it/s]

Average training loss: 0.0768
Validation Accuracy: 0.9955, F1 Score: 0.9955


Epoch 3/3:   0%|          | 0/2582 [00:00<?, ?it/s]

Average training loss: 0.0396
Validation Accuracy: 0.9962, F1 Score: 0.9962
Training completed for all folds.


In [3]:
# 레이블 인코딩
print("레이블 인코딩 중...")
le = LabelEncoder()
y = le.fit_transform(y)
print(f"고유 감정 레이블 수: {len(le.classes_)}")

# 마침표와 쉼표를 제거하는 함수 정의
def remove_punctuation(text):
    return text.replace('.', '').replace(',', '')

# 'text' 열에 함수 적용
X = X.apply(remove_punctuation)

print(X)

# 불용어 목록 정의
stopwords = set([
    '는', '은', '이', '가', '을', '를', '에', '의', '도', '에서', '와', '과', '로', '으로',
    '그리고', '그러나', '그런데', '하지만', '또는', '그래서', '따라서',
    '이것', '저것', '그것', '우리', '여러분', '나', '너', '그', '그녀',
    '입니다', '있습니다', '합니다', '되었습니다', '있다', '것이다', '합니다'
])

# 불용어 제거 함수 정의
def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()
        meaningful_words = [word for word in words if word not in stopwords]
        return ' '.join(meaningful_words)
    return text

# 'text' 열에 함수 적용
X = X.apply(remove_punctuation)

print(X)


# 데이터 분할
print("데이터 분할 중...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"학습 데이터 수: {len(X_train)}, 테스트 데이터 수: {len(X_test)}")

# KLUE/BERT 모델 및 토크나이저 로드
print("KLUE/BERT 모델 및 토크나이저 로드 중...")
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

# GPU 사용 가능 시 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = bert_model.to(device)
print(f"모델 로드 완료. 사용 중인 디바이스: {device}")


# 데이터 토큰화 및 모델 입력 준비 함수
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# PyTorch 텐서를 numpy 배열로 변환하는 함수
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

# BERT 임베딩을 생성하는 함수 (배치 처리)
def get_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="임베딩 생성 중"):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.append(embeddings)
    return np.vstack(all_embeddings)

# 학습 및 테스트 데이터에 대한 BERT 임베딩 생성
print("학습 데이터 임베딩 생성 중...")
start_time = time.time()
X_train_emb = get_embeddings(X_train.tolist())
print(f"학습 데이터 임베딩 생성 완료. 소요 시간: {time.time() - start_time:.2f}초")

print("테스트 데이터 임베딩 생성 중...")
start_time = time.time()
X_test_emb = get_embeddings(X_test.tolist())
print(f"테스트 데이터 임베딩 생성 완료. 소요 시간: {time.time() - start_time:.2f}초")


CSV 파일 로드 중...
데이터 로드 완료. 샘플 수: 16137
레이블 인코딩 중...
고유 감정 레이블 수: 7
0                                          헐! 나 이벤트에 당첨 됐어
1                내가 좋아하는 인플루언서가 이벤트를 하더라고 그래서 그냥 신청 한번 해봤지
2                                한 명 뽑는 거였는데 그게 바로 내가 된 거야
3        당연히 마음에 드는 선물이니깐 이벤트에 내가 신청 한번 해본 거지 비싼 거야 그래서...
4                           에피타이저 정말 좋아해 그 것도 괜찮은 생각인 것 같애
                               ...                        
16132                                   좋은 생각이야 괜찮은 운동 있어?
16133                                       어떤 홈 트레이닝이 있어?
16134                            맞아 나한테만 퉁명스럽고 일을 많이 시키더라고
16135                                    하지만 기분이 나쁜 걸 어떡해?
16136                                     자취방 엘리베이턴데 정전인가봐
Name: 발화문, Length: 16137, dtype: object
0                                          헐! 나 이벤트에 당첨 됐어
1                내가 좋아하는 인플루언서가 이벤트를 하더라고 그래서 그냥 신청 한번 해봤지
2                                한 명 뽑는 거였는데 그게 바로 내가 된 거야
3        당연히 마음에 드는 선물이니깐 이벤트에 내가 신청 한번 해본 거지 비싼 거야 그래서...
4        

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


모델 로드 완료. 사용 중인 디바이스: cuda
학습 데이터 임베딩 생성 중...


임베딩 생성 중: 100%|██████████| 404/404 [00:16<00:00, 25.11it/s]


학습 데이터 임베딩 생성 완료. 소요 시간: 16.10초
테스트 데이터 임베딩 생성 중...


임베딩 생성 중: 100%|██████████| 101/101 [00:03<00:00, 25.33it/s]

테스트 데이터 임베딩 생성 완료. 소요 시간: 3.99초


In [4]:
import plotly.express as px
from sklearn.manifold import TSNE

# 3차원 t-SNE
tsne_3d = TSNE(n_components=3, random_state=42)
X_tsne_3d = tsne_3d.fit_transform(X_train_emb)

# 데이터프레임 생성
df = pd.DataFrame({
    'tsne-3d-1': X_tsne_3d[:,0],
    'tsne-3d-2': X_tsne_3d[:,1],
    'tsne-3d-3': X_tsne_3d[:,2],
    'emotion': [le.classes_[i] for i in y_train],
    'text': X_train.tolist()
})

# 3D 산점도 생성
fig = px.scatter_3d(df, x='tsne-3d-1', y='tsne-3d-2', z='tsne-3d-3',
                    color='emotion', hover_data=['text'], labels={'color': 'emotion'})
fig.update_layout(title='3D t-SNE of Emotion Embeddings')
fig.show()